In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb 
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [2]:
print("######读取数据（基于EEG）######")
#EEG特征表
eegFeatureDF = pickle.load(open("./dump_file/eegFeatureDF","rb"))
#GSR特征表
GSR_feature_df = pickle.load(open("./dump_file/df_feat_selected","rb"))
all_df_y_valence = pickle.load(open("./dump_file/all_df_y_valence","rb"))
all_df_y = pickle.load(open("./dump_file/all_df_y","rb"))
all_df_y_2c = pickle.load(open("./dump_file/all_df_y_2c","rb"))
print("eegFeatureDF.shape:",eegFeatureDF.shape)

######读取数据（基于EEG）######
eegFeatureDF.shape: (1280, 512)


In [3]:
'''
print("######数据缩放处理，归一化处理######")
min_max_scaler = MinMaxScaler()
eegFeatureDF = min_max_scaler.fit_transform(eegFeatureDF)
'''

'\nprint("######数据缩放处理，归一化处理######")\nmin_max_scaler = MinMaxScaler()\neegFeatureDF = min_max_scaler.fit_transform(eegFeatureDF)\n'

In [4]:
#featureDF = pd.concat([eegFeatureDF,GSR_feature_df],axis=1)
#featureDF = GSR_feature_df
featureDF = eegFeatureDF

In [12]:
#############################下面用愉悦度（效价）做分类#################################
if True:
    print("----------------这是高低愉悦度度二分类预测----------------")
    data = featureDF
    target = all_df_y_2c #高低愉悦度
    
    print("######xgb classification CV######")
    xgb_model = xgb.XGBClassifier(max_depth=5,learning_rate=0.1,n_estimators=50,
                                      objective='binary:logistic',booster='gbtree',n_jobs=10,
                                      subsample=0.9, colsample_bytree=0.9, colsample_bylevel=0.9,
                                      reg_alpha=0.5, reg_lambda=1.0,gamma=0,
                                      scale_pos_weight=1)
    xgb_scores = cross_val_score(xgb_model,data,target,cv=5,scoring='accuracy')
    print("xgb_scores:",abs(xgb_scores))
    print("xgb_scores_mean:",abs(xgb_scores.mean()))


----------------这是高低愉悦度度二分类预测----------------
######xgb classification CV######
xgb_scores: [0.61089494 0.59922179 0.55078125 0.52941176 0.61176471]
xgb_scores_mean: 0.580414890421149
